In [ ]:
# -*- coding: utf-8 -*-
"""
This is the main module to construct a magnetohydrostatic solar atmosphere,
given a specified magnetic network of self-similar magnetic flux tubes and
save the output to gdf format.

To select an existing configuration change the import as model_pars, set Nxyz,
xyz_SI and any other special parameters, then execute mhs_atmopshere.

To add new configurations:
add the model options to set_options in parameters/options.py;
add options required in parameters/model_pars.py;
add alternative empirical data sets to hs_model/;
add alternativ table than interploate_atmosphere in hs_model/hs_atmosphere.py;
add option to get_flux_tubes in mhs_model/flux_tubes.py

If an alternative formulation of the flux tube is required add options to
construct_magnetic_field and construct_pairwise_field in
mhs_model/flux_tubes.py

Plotting options are included in plot/mhs_plot.py
"""

In [1]:


import os
import numpy as np
import pysac.mhs_atmosphere as atm
import astropy.units as u
from pysac.mhs_atmosphere.parameters.model_pars import paper2d as model_pars

In [2]:
l_mpi = False
rank = 0
size = 1
    



In [3]:
#==============================================================================
#set up model parameters
#==============================================================================
local_procs=1
#standard set of logical switches
option_pars = atm.options.set_options(model_pars, l_mpi, l_gdf=True)
#standard conversion to dimensionless units and physical constants
scales, physical_constants = \
    atm.units_const.get_parameters()

#obtain code coordinates and model parameters in astropy units
coords = atm.model_pars.get_coords(model_pars['Nxyz'], u.Quantity(model_pars['xyz']))

#interpolate the hs 1D profiles from empirical data source[s]
empirical_data = atm.hs_atmosphere.read_VAL3c_MTW(mu=physical_constants['mu'])

table = \
        atm.hs_atmosphere.interpolate_atmosphere(empirical_data,
                                   coords['Zext']
                                  )
if model_pars['model'] == 'mfe_setup':
    table['rho'] = table['rho'] + table['rho'].min()*3.6

In [4]:
#==============================================================================
#calculate 1d hydrostatic balance from empirical density profile
#==============================================================================
# the hs pressure balance is enhanced by pressure equivalent to the
# residual mean coronal magnetic pressure contribution once the magnetic
# field has been applied
magp_meanz = np.ones(len(coords['Z'])) * u.one
magp_meanz *= model_pars['pBplus']**2/(2*physical_constants['mu0'])

pressure_Z, rho_Z, Rgas_Z = atm.hs_atmosphere.vertical_profile(
                                                 coords['Z'],
                                                 table,
                                                 magp_meanz,
                                                 physical_constants,
                                                 coords['dz']
                                                 )

In [5]:
#==============================================================================
# load flux tube footpoint parameters
#==============================================================================
# axial location and value of Bz at each footpoint
model_pars['B_corona']/=model_pars['nftubes']
xi, yi, Si = atm.flux_tubes.get_flux_tubes(
                                model_pars,
                                coords,
                                option_pars
                               )
#==============================================================================
# split domain into processes if mpi
#==============================================================================
ax, ay, az = np.mgrid[coords['xmin']:coords['xmax']:1j*model_pars['Nxyz'][0],
                      coords['ymin']:coords['ymax']:1j*model_pars['Nxyz'][1],
                      coords['zmin']:coords['zmax']:1j*model_pars['Nxyz'][2]]

In [6]:
x, y, z = ax, ay, az

x = u.Quantity(x, unit=coords['xmin'].unit)
y = u.Quantity(y, unit=coords['ymin'].unit)
z = u.Quantity(z, unit=coords['zmin'].unit)
#==============================================================================
# initialize zero arrays in which to add magnetic field and mhs adjustments
#==============================================================================
Bx   = u.Quantity(np.zeros(x.shape), unit=u.T)  # magnetic x-component
By   = u.Quantity(np.zeros(x.shape), unit=u.T)  # magnetic y-component
Bz   = u.Quantity(np.zeros(x.shape), unit=u.T)  # magnetic z-component
pressure_m = u.Quantity(np.zeros(x.shape), unit=u.Pa) # magneto-hydrostatic adjustment to pressure
rho_m = u.Quantity(np.zeros(x.shape), unit=u.kg/u.m**3)      # magneto-hydrostatic adjustment to density
# initialize zero arrays in which to add balancing forces and magnetic tension
Fx   = u.Quantity(np.zeros(x.shape), unit=u.N/u.m**3)  # balancing force x-component
Fy   = u.Quantity(np.zeros(x.shape), unit=u.N/u.m**3)  # balancing force y-component
# total tension force for comparison with residual balancing force
Btensx  = u.Quantity(np.zeros(x.shape), unit=u.N/u.m**3)
Btensy  = u.Quantity(np.zeros(x.shape), unit=u.N/u.m**3)

In [7]:
#==============================================================================
#calculate the magnetic field and pressure/density balancing expressions
#==============================================================================
for i in range(0,model_pars['nftubes']):
    for j in range(i,model_pars['nftubes']):
        if rank == 0:
            print'calculating ij-pair:',i,j
        if i == j:
            pressure_mi, rho_mi, Bxi, Byi ,Bzi, B2x, B2y =\
                atm.flux_tubes.construct_magnetic_field(
                                             x, y, z,
                                             xi[i], yi[i], Si[i],
                                             model_pars, option_pars,
                                             physical_constants,
                                             scales
                                            )
            Bx, By, Bz = Bxi+Bx, Byi+By ,Bzi+Bz
            Btensx += B2x
            Btensy += B2y
            pressure_m += pressure_mi
            rho_m += rho_mi
        else:
            pressure_mi, rho_mi, Fxi, Fyi, B2x, B2y =\
                atm.flux_tubes.construct_pairwise_field(
                                             x, y, z,
                                             xi[i], yi[i],
                                             xi[j], yi[j], Si[i], Si[j],
                                             model_pars,
                                             option_pars,
                                             physical_constants,
                                             scales
                                            )
            pressure_m += pressure_mi
            rho_m += rho_mi
            Fx   += Fxi
            Fy   += Fyi
            Btensx += B2x
            Btensy += B2y

# clear some memory
del pressure_mi, rho_mi, Bxi, Byi ,Bzi, B2x, B2y

calculating ij-pair: 0 0


/home/mike/proj/pysac/pysac/mhs_atmosphere/mhs_model/flux_tubes.py:312: Warning: pbbal.max() = 90.0342248086 kg / (m s2)
  warnings.warn("pbbal.max() = {}".format(pbbal.max().decompose()), Warning)


calculating ij-pair: 0 1
pbbal.max() =  0.0693537676407 A2 T2 / N
calculating ij-pair: 0 2
pbbal.max() =  0.0693486540176 A2 T2 / N
calculating ij-pair: 0 3
pbbal.max() =  0.0693144131854 A2 T2 / N
calculating ij-pair: 0 4
pbbal.max() =  0.069295036155 A2 T2 / N
calculating ij-pair: 0 5
pbbal.max() =  0.0692966508168 A2 T2 / N
calculating ij-pair: 0 6
pbbal.max() =  0.0691477853241 A2 T2 / N
calculating ij-pair: 0 7
pbbal.max() =  0.0691483195345 A2 T2 / N
calculating ij-pair: 0 8
pbbal.max() =  0.0691185421353 A2 T2 / N
calculating ij-pair: 0 9
pbbal.max() =  0.0689641264173 A2 T2 / N
calculating ij-pair: 0 10
pbbal.max() =  0.070113671602 A2 T2 / N
calculating ij-pair: 0 11
pbbal.max() =  0.0689502006122 A2 T2 / N
calculating ij-pair: 0 12
pbbal.max() =  0.1264285813 A2 T2 / N
calculating ij-pair: 0 13
pbbal.max() =  0.12463396894 A2 T2 / N
calculating ij-pair: 0 14
pbbal.max() =  0.132281540778 A2 T2 / N
calculating ij-pair: 0 15
pbbal.max() =  0.0701789709186 A2 T2 / N
calculating 

/home/mike/proj/pysac/pysac/mhs_atmosphere/mhs_model/flux_tubes.py:312: Warning: pbbal.max() = 88.9991040179 kg / (m s2)
  warnings.warn("pbbal.max() = {}".format(pbbal.max().decompose()), Warning)


calculating ij-pair: 1 2
pbbal.max() =  0.0693610157485 A2 T2 / N
calculating ij-pair: 1 3
pbbal.max() =  0.0693485425528 A2 T2 / N
calculating ij-pair: 1 4
pbbal.max() =  0.0693327039021 A2 T2 / N
calculating ij-pair: 1 5
pbbal.max() =  0.0693317364498 A2 T2 / N
calculating ij-pair: 1 6
pbbal.max() =  0.0692404947013 A2 T2 / N
calculating ij-pair: 1 7
pbbal.max() =  0.069237507595 A2 T2 / N
calculating ij-pair: 1 8
pbbal.max() =  0.0692190396443 A2 T2 / N
calculating ij-pair: 1 9
pbbal.max() =  0.0688973016546 A2 T2 / N
calculating ij-pair: 1 10
pbbal.max() =  0.0688983877151 A2 T2 / N
calculating ij-pair: 1 11
pbbal.max() =  0.0688963639765 A2 T2 / N
calculating ij-pair: 1 12
pbbal.max() =  0.11685052725 A2 T2 / N
calculating ij-pair: 1 13
pbbal.max() =  0.114141244893 A2 T2 / N
calculating ij-pair: 1 14
pbbal.max() =  0.120769573332 A2 T2 / N
calculating ij-pair: 1 15
pbbal.max() =  0.0688983561537 A2 T2 / N
calculating ij-pair: 1 16
pbbal.max() =  0.131617809127 A2 T2 / N
calculati

...

/home/mike/proj/pysac/pysac/mhs_atmosphere/mhs_model/flux_tubes.py:312: Warning: pbbal.max() = 90.2595742968 kg / (m s2)
  warnings.warn("pbbal.max() = {}".format(pbbal.max().decompose()), Warning)


calculating ij-pair: 15 16
pbbal.max() =  0.0690344126573 A2 T2 / N
calculating ij-pair: 15 17
pbbal.max() =  0.0690301653587 A2 T2 / N
calculating ij-pair: 16 16


/home/mike/proj/pysac/pysac/mhs_atmosphere/mhs_model/flux_tubes.py:312: Warning: pbbal.max() = 90.1155859501 kg / (m s2)
  warnings.warn("pbbal.max() = {}".format(pbbal.max().decompose()), Warning)


calculating ij-pair: 16 17
pbbal.max() =  0.0693632529976 A2 T2 / N
calculating ij-pair: 17 17


/home/mike/proj/pysac/pysac/mhs_atmosphere/mhs_model/flux_tubes.py:312: Warning: pbbal.max() = 89.3075414857 kg / (m s2)
  warnings.warn("pbbal.max() = {}".format(pbbal.max().decompose()), Warning)


In [8]:
print asc

NameError: name 'asc' is not defined

In [ ]:
#==============================================================================
# Construct 3D hs arrays and then add the mhs adjustments to obtain atmosphere
#==============================================================================
# select the 1D array spanning the local mpi process; the add/sub of dz to
# ensure all indices are used, but only once
indz = np.where(coords['Z'] >= z.min()-0.1*coords['dz']) and \
       np.where(coords['Z'] <= z.max()+0.1*coords['dz'])
pressure_z, rho_z, Rgas_z = pressure_Z[indz], rho_Z[indz], Rgas_Z[indz]
# local proc 3D mhs arrays
pressure, rho = atm.mhs_3D.mhs_3D_profile(z,
                                   pressure_z,
                                   rho_z,
                                   pressure_m,
                                   rho_m
                                  )
magp = (Bx**2 + By**2 + Bz**2)/(2.*physical_constants['mu0'])
if rank ==0:
    print'max B corona = ',magp[:,:,-1].max().decompose()
    print'min B corona = ',magp[:,:,-1].min().decompose()
energy = atm.mhs_3D.get_internal_energy(pressure,
                                                  magp,
                                                  physical_constants)

In [ ]:
#============================================================================
# Save data for SAC and plotting
#============================================================================
# set up data directory and file names
# may be worthwhile locating on /data if files are large
datadir = os.path.expanduser('~/Documents/mhs_atmosphere/'+model_pars['model']+'/')
filename = datadir + model_pars['model'] + option_pars['suffix']
if not os.path.exists(datadir):
    os.makedirs(datadir)
sourcefile = datadir + model_pars['model'] + '_sources' + option_pars['suffix']
aux3D = datadir + model_pars['model'] + '_3Daux' + option_pars['suffix']
aux1D = datadir + model_pars['model'] + '_1Daux' + option_pars['suffix']
# save the variables for the initialisation of a SAC simulation
atm.mhs_snapshot.save_SACvariables(
              filename,
              rho,
              Bx,
              By,
              Bz,
              energy,
              option_pars,
              physical_constants,
              coords,
              model_pars['Nxyz']
             )
# save the balancing forces as the background source terms for SAC simulation
atm.mhs_snapshot.save_SACsources(
              sourcefile,
              Fx,
              Fy,
              option_pars,
              physical_constants,
              coords,
              model_pars['Nxyz']
             )
# save auxilliary variable and 1D profiles for plotting and analysis
Rgas = u.Quantity(np.zeros(x.shape), unit=Rgas_z.unit)
Rgas[:] = Rgas_z
temperature = pressure/rho/Rgas
if not option_pars['l_hdonly']:
    inan = np.where(magp <=1e-7*pressure.min())
    magpbeta = magp
    magpbeta[inan] = 1e-7*pressure.min()  # low pressure floor to avoid NaN
    pbeta  = pressure/magpbeta
else:
    pbeta  = magp+1.0    #dummy to avoid NaN
alfven = np.sqrt(2.*magp/rho)
#if rank == 0:
#    print'Alfven speed Z.min to Z.max =',\
#    alfven[model_pars['Nxyz'][0]/2,model_pars['Nxyz'][1]/2, 0].decompose(),\
#    alfven[model_pars['Nxyz'][0]/2,model_pars['Nxyz'][1]/2,-1].decompose()
cspeed = np.sqrt(physical_constants['gamma']*pressure/rho)


In [ ]:
atm.mhs_snapshot.save_auxilliary3D(
              aux3D,
              pressure_m,
              rho_m,
              temperature,
              pbeta,
              alfven,
              cspeed,
              Btensx,
              Btensy,
              option_pars,
              physical_constants,
              coords,
              model_pars['Nxyz']
             )
atm.mhs_snapshot.save_auxilliary1D(
              aux1D,
              pressure_Z,
              rho_Z,
              Rgas_Z,
              option_pars,
              physical_constants,
              coords,
              model_pars['Nxyz']
             )
if rho.min()<0 or pressure.min()<0:
    print"FAIL: negative rho.min() {} and/or pressure.min() {}.".format(
    rho.min(),pressure.min())
FWHM = 2*np.sqrt(np.log(2))*model_pars['radial_scale']
print'FWHM(0) =',FWHM